In [1]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

fname = "root://eospublic.cern.ch//eos/opendata/cms/Run2016H/DoubleMuon/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v1/2510000/127C2975-1B1C-A046-AABF-62B77E757A86.root"
#fname = "root://localhost//disk03/opendata_cms/DoubleMuon/Run2016H-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD/127C2975-1B1C-A046-AABF-62B77E757A86.root"
events = NanoEventsFactory.from_root(
    {fname: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DoubleMuon"},
).events()

print("Number of events = ", ak.num(events.Muon, axis=0).compute())

/usr/local/venv/lib/python3.10/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


Number of events =  2147195


In [2]:
print("Number of muons = ", ak.count(events.Muon.mass, axis=None).compute())

Number of muons =  4806013


In [3]:
selected_muons = events.Muon[(events.Muon.isGlobal == 1)]
print("Number of selected muons after isGlobal==1: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after isGlobal==1:  4110800


In [4]:
selected_muons = selected_muons[(selected_muons.nStations > 0)]
print("Number of selected muons after nStations>0: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after nStations>0:  3936437


In [5]:
selected_muons = selected_muons[(selected_muons.nTrackerLayers > 5)]
print("Number of selected muons after nTrackerLayers>5: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after nTrackerLayers>5:  3908889


In [6]:
selected_muons = selected_muons[(selected_muons.dz < 0.3) & (selected_muons.dxy < 20.0)]
print("Number of selected muons after PV impact param sel: ", ak.count(selected_muons.mass, axis=None).compute())       

Number of selected muons after PV impact param sel:  3698722


In [8]:
selected_muons = selected_muons[(selected_muons.pt > 3.0) & (abs(selected_muons.eta) < 2.4)]
print("Number of selected muons after pt,eta sel: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after pt,eta sel:  3690368
